### Libs

In [1]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
from io import BytesIO
from PIL import Image
from pptx import Presentation
from pptx.util import Inches
import logging
import os
from ipywidgets import interact, interactive
from IPython.display import HTML

### Data - Create the multi-index DataFrame generator

In [2]:
def create_multi_index_dataframe(udl, matu, param, level, start_date='2023-01-01', periods=1000, freq='D'):
    arrays = []
    for u in udl:
        for m in matu:
            for p in param:
                for l in level[p]:
                    arrays.append((u, m, p, l))
    multi_index = pd.MultiIndex.from_tuples(arrays, names=["UDL", "Matu", "Param", "Level"])
    time_index = pd.date_range(start=start_date, periods=periods, freq=freq)
    base_data = np.linspace(1, 100, len(time_index))
    data = np.array([base_data * (1 + 0.01 * i) + np.random.normal(0, 1, len(time_index)) for i in range(len(multi_index))]).T
    df = pd.DataFrame(data=data, index=time_index, columns=multi_index)
    return df

### Widget Manager Class

In [3]:
class WidgetManager:
    def __init__(self, df):
        self.plot_type_widget = widgets.Dropdown(options=[], value=None, description="Plot Type:", layout=widgets.Layout(width='100%'))
        self.udl_widget = widgets.SelectMultiple(
            options=df.columns.get_level_values('UDL').unique(),
            value=[df.columns.get_level_values('UDL').unique()[0]],
            description='UDL:',
            layout=widgets.Layout(height='100px', width='100%')
        )
        self.matu_widget = widgets.Dropdown(
            options=df.columns.get_level_values('Matu').unique(),
            value=df.columns.get_level_values('Matu').unique()[0],
            description='Matu:',
            layout=widgets.Layout(height='40px', width='100%')
        )
        self.param_widget = widgets.SelectMultiple(
            options=df.columns.get_level_values('Param').unique(),
            value=[df.columns.get_level_values('Param').unique()[0]],
            description='Param:',
            layout=widgets.Layout(height='100px', width='100%')
        )
        self.level_widget = widgets.SelectMultiple(
            options=df.columns.get_level_values('Level').unique(),
            value=[df.columns.get_level_values('Level').unique()[0]],
            description='Level:',
            layout=widgets.Layout(height='100px', width='100%')
        )
        self.window_widget = widgets.Dropdown(options=[52, 104, 156], value=52, description='Window:', layout=widgets.Layout(height='40px', width='100%'))
        self.start_date_widget = widgets.DatePicker(description='Start Date:', value=pd.Timestamp(df.index.min()).to_pydatetime(), layout=widgets.Layout(width='100%'))
        self.end_date_widget = widgets.DatePicker(description='End Date:', value=pd.Timestamp(df.index.max()).to_pydatetime(), layout=widgets.Layout(width='100%'))
        self.plot_button = widgets.Button(description='Preview Plot', button_style='success', layout=widgets.Layout(width='28%', min_width='120px'))
        self.add_to_slide_button = widgets.Button(description='Add to Slide', button_style='info', disabled=True, layout=widgets.Layout(width='28%', min_width='120px'))
        self.export_button = widgets.Button(description="Export to PPT", button_style='warning', layout=widgets.Layout(width='28%', min_width='120px'))
        self.position_dropdown = widgets.Dropdown(options=["top-left", "top-right", "bottom-left", "bottom-right"], description="Position:", layout=widgets.Layout(height='40px', width='100%'))
        self.add_slide_button = widgets.Button(description="Add Slide", button_style="primary", layout=widgets.Layout(width='28%', min_width='120px'))

### Plot Manager to Register Plots and Manage Widgets

In [4]:
class PlotManager:
    def __init__(self):
        self.plots = {}

    def register_plot(self, name, plot_function, required_widgets):
        """Register a new plot type."""
        self.plots[name] = {
            "function": plot_function,
            "widgets": required_widgets
        }

    def get_plot_function(self, name):
        return self.plots.get(name, {}).get("function")

    def get_required_widgets(self, name):
        return self.plots.get(name, {}).get("widgets", [])

### Registering Plot Functions

In [5]:
def create_time_series_plot(filtered_df, window):
    result_df = filtered_df.rolling(window=window).mean().fillna(0)
    fig, ax = plt.subplots(figsize=FIGURE_SIZE)
    for column in result_df.columns:
        ax.plot(result_df.index, result_df[column], label=str(column))
    ax.set_title("Time Series Plot Preview")
    ax.legend()
    return fig

def create_stats_chart(filtered_df):
    fig, ax = plt.subplots(figsize=FIGURE_SIZE)
    categories = filtered_df.columns.get_level_values('UDL').unique()
    min_values = filtered_df.min().values
    max_values = filtered_df.max().values
    avg_values = filtered_df.mean().values
    last_values = filtered_df.iloc[-1].values
    percentile_20 = filtered_df.quantile(0.2).values
    percentile_80 = filtered_df.quantile(0.8).values

    colors = {
        "percentile_range": "gray",
        "min_max": "black",
        "avg": "green",
        "last": "red"
    }
    icon_width = 0.3

    for i, category in enumerate(categories):
        ax.bar(i, percentile_80[i] - percentile_20[i], bottom=percentile_20[i], color=colors['percentile_range'], alpha=0.5, edgecolor='none', width=icon_width)
        ax.plot([i - icon_width / 2, i + icon_width / 2], [min_values[i], min_values[i]], color=colors['min_max'], linewidth=3)
        ax.plot([i - icon_width / 2, i + icon_width / 2], [max_values[i], max_values[i]], color=colors['min_max'], linewidth=3)
        ax.plot(i, avg_values[i], marker='^', color=colors['avg'], markersize=10, markeredgewidth=1.5, markeredgecolor='black')
        ax.plot(i, last_values[i], marker='D', color=colors['last'], markersize=10, markeredgewidth=1.5, markeredgecolor='black')

    ax.set_xticks(range(len(categories)))
    ax.set_xticklabels(categories)
    ax.axhline(0, color='black', linewidth=2.0)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    plt.tight_layout()
    return fig

### Utility function to generate plot image

In [6]:
def generate_plot_image(fig, ax):
    img_stream = BytesIO()
    fig.savefig(img_stream, format='png')
    img_stream.seek(0)
    img = Image.open(img_stream)
    ax.imshow(img, aspect='auto')
    ax.set_xticks([])
    ax.set_yticks([])

### Slide management class

In [7]:
class SlideManager:
    def __init__(self):
        self.slides = [{"plots": []}]
        self.selected_slide_index = 0

    def add_slide(self):
        self.slides.append({"plots": []})
        self.selected_slide_index = len(self.slides) - 1

    def add_plot_to_slide(self, plot_data, position):
        slide_index = self.selected_slide_index
        # Remove any existing plot in the selected position
        self.slides[slide_index]["plots"] = [plot for plot in self.slides[slide_index]["plots"] if plot["position"] != position]
        # Add the new plot to the selected position
        self.slides[slide_index]["plots"].append(plot_data)

    def get_current_slide(self):
        return self.slides[self.selected_slide_index]

To modularize and scale the given App class effectively, we can focus on separating concerns and extracting repetitive or tightly coupled logic into smaller, reusable functions or classes. Here's how to approach this:

#### Goals:
- Modularization: Break down monolithic methods into smaller functions or classes.
- Scalability: Allow for easy addition of new widgets, plots, and layout configurations.
- Readability and Maintainability: Improve code clarity by organizing related logic into separate modules or classes.

#### Key Areas for Refactoring:
- Plot Registration and Display Logic: Abstract plot handling into a dedicated handler class.
- Data Filtering Logic: Encapsulate filtering in a standalone utility.
- Widget Binding: Separate widget event bindings into modular functions or classes.
- Layout Management: Create reusable layout constructors for different parts of the interface.
Here’s the refactored structure:

### Abstract Plot Handling

######## Extract plot-related logic into a PlotHandler class to manage the PlotManager and associated methods.
class PlotHandler:
    def __init__(self, df, widgets, plot_manager, display_area, output_area):
        self.df = df
        self.widgets = widgets
        self.plot_manager = plot_manager
        self.display_area = display_area
        self.output_area = output_area
        self.current_plot_data = {}

    def filter_data(self):
        selected_udl = list(self.widgets.udl_widget.value)
        selected_matu = (
            [self.widgets.matu_widget.value]
            if isinstance(self.widgets.matu_widget, widgets.Dropdown)
            else list(self.widgets.matu_widget.value)
        )
        selected_param = list(self.widgets.param_widget.value)
        selected_level = list(self.widgets.level_widget.value)
        start_date = self.widgets.start_date_widget.value
        end_date = self.widgets.end_date_widget.value

        if start_date > end_date:
            raise ValueError("Start Date must be before End Date.")

        filtered_df = self.df.loc[
            start_date:end_date,
            (self.df.columns.get_level_values('UDL').isin(selected_udl)) &
            (self.df.columns.get_level_values('Matu').isin(selected_matu)) &
            (self.df.columns.get_level_values('Param').isin(selected_param)) &
            (self.df.columns.get_level_values('Level').isin(selected_level))
        ]
        return filtered_df

    def generate_plot(self, plot_type, filtered_df):
        plot_function = self.plot_manager.get_plot_function(plot_type)
        if not plot_function:
            raise ValueError(f"Plot type '{plot_type}' is not registered.")

        window = self.widgets.window_widget.value
        if plot_type == "Time Series Plot":
            return plot_function(filtered_df, window)
        return plot_function(filtered_df)

    def update_plot(self, b):
        plot_type = self.widgets.plot_type_widget.value
        try:
            filtered_df = self.filter_data()
            if filtered_df.empty:
                raise ValueError("No valid data available for the selected combination.")

            fig = self.generate_plot(plot_type, filtered_df)
            self.display_area.clear_output(wait=True)
            self.output_area.clear_output(wait=True)
            with self.display_area:
                plt.show(fig)
            self.current_plot_data = {"figure": fig, "data": filtered_df, "type": plot_type}
        except Exception as e:
            with self.output_area:
                clear_output(wait=True)
                print(f"Error: {e}")

### Encapsulate Data Filtering

######## Create a standalone utility function or class for data filtering. This will simplify the plot logic and allow reuse elsewhere.
class DataFilter:
    @staticmethod
    def filter(df, udl, matu, param, level, start_date, end_date):
        return df.loc[
            start_date:end_date,
            (df.columns.get_level_values('UDL').isin(udl)) &
            (df.columns.get_level_values('Matu').isin(matu)) &
            (df.columns.get_level_values('Param').isin(param)) &
            (df.columns.get_level_values('Level').isin(level))
        ]

### Widget Event Binding

######## Extract event binding logic to a separate class for better maintainability.
class EventBinder:
    def __init__(self, widgets, plot_handler):
        self.widgets = widgets
        self.plot_handler = plot_handler

    def bind_events(self):
        self.widgets.plot_type_widget.observe(self.on_plot_type_change, names='value')
        self.widgets.plot_button.on_click(self.plot_handler.update_plot)

    def on_plot_type_change(self, change):
        selected_plot = change['new']
        required_widgets = self.plot_handler.plot_manager.get_required_widgets(selected_plot)

        for widget_name, widget in vars(self.widgets).items():
            if widget_name in required_widgets:
                widget.layout.display = 'flex'
                widget.disabled = False
            else:
                widget.layout.display = 'none'
                widget.disabled = True

### Refactored App Class

####### The App class coordinate the PlotHandler, EventBinder, and LayoutBuilder.
class App:
    def __init__(self, df):
        self.df = df
        self.widgets = WidgetManager(df)
        self.plot_manager = PlotManager()
        self.plot_handler = PlotHandler(df, self.widgets, self.plot_manager, widgets.Output(), widgets.Output())
        self.event_binder = EventBinder(self.widgets, self.plot_handler)

        self.slide_manager = SlideManager()
        self.left_sidebar = widgets.VBox()
        self.main_display = widgets.Output(layout=widgets.Layout(width="70%", height="500px"))
        self.df_below_output = widgets.Output(layout=widgets.Layout(width="30%"))
        self.customization_window = LayoutBuilder.create_control_buttons(self.widgets)

        self.build_layout()
        self.event_binder.bind_events()

    def build_layout(self):
        layout = LayoutBuilder.create_main_layout(
            self.left_sidebar,
            self.main_display,
            self.customization_window,
            self.df_below_output
        )
        display(layout)

### class LayoutBuilder

In [8]:
class LayoutBuilder:
    @staticmethod
    def create_customization_window(widget_manager):
        customization_header = widgets.HTML(
            value="<h3 style='color: #333; font-family: Arial, sans-serif;'>Plot Customization</h3>"
        )
        control_buttons = LayoutBuilder.create_control_buttons(widget_manager)
        customization_widgets = [
            customization_header,
            widget_manager.plot_type_widget,
            widget_manager.udl_widget,
            widget_manager.matu_widget,
            widget_manager.param_widget,
            widget_manager.level_widget,
            widget_manager.window_widget,
            widget_manager.start_date_widget,
            widget_manager.end_date_widget,
            widget_manager.position_dropdown,
            control_buttons,
        ]
        return widgets.VBox(
            customization_widgets,
            layout=widgets.Layout(
                width="30%",
                padding="15px",
                border="1px solid #d3d3d3",
                background_color="#f9f9f9"
            )
        )

    @staticmethod
    def create_control_buttons(widget_manager):
        control_buttons = [
            widget_manager.add_slide_button,
            widget_manager.export_button,
            widget_manager.plot_button,
            widget_manager.add_to_slide_button
        ]
        return widgets.GridBox(
            control_buttons,
            layout=widgets.Layout(
                grid_template_columns="repeat(2, 48%)",
                grid_gap="5px",
                width="100%",
                min_height="60px"
            )
        )

    @staticmethod
    def create_main_layout(left_sidebar, main_display, customization_window, df_below_output):
        main_content = widgets.HBox(
            [left_sidebar, main_display, customization_window],
            layout=widgets.Layout(align_items='flex-start')
        )
        return widgets.VBox(
            [main_content, df_below_output],
            layout=widgets.Layout(width="100%")
        )

### Helper Functions

#### TO DO
- filter_dataframe - make it handl eanything gracefully
- display_dataframe_below - filter_dataframe
- add debug print below?
- create_output_widget - reactivity
- bind_widget_events - reactivity
- update_widget_visibility - reactivity
- handle_plot_button_clicked - reactivity

- register_available_plots - feed

#### DONE
- handle_add_to_slide
- handle_add_slide
- handle_export
- switch_to_slide_view_helper
- update_sidebar_helper.
- create_sidebar_widget

In [18]:
def create_output_widget(width, height=None, border=None, padding=None, margin=None):
    return widgets.Output(layout=widgets.Layout(width=width, height=height, border=border, padding=padding, margin=margin))

def create_sidebar_widget(width, border, padding, background_color, margin):
    return widgets.VBox(layout=widgets.Layout(width=width, border=border, padding=padding, background_color=background_color, margin=margin))

def bind_widget_events(widgets_manager, event_mapping):
    for widget_name, handler in event_mapping.items():
        widget = getattr(widgets_manager, widget_name)
        if hasattr(widget, 'on_click'):
            widget.on_click(handler)
        elif hasattr(widget, 'observe'):
            widget.observe(handler, names='value')

def register_available_plots(plot_manager, widgets_manager):
    plot_manager.register_plot("Time Series Plot", create_time_series_plot, [
        "udl_widget", "matu_widget", "param_widget", "window_widget", "start_date_widget", "end_date_widget"
    ])
    plot_manager.register_plot("Custom Stats Chart", create_stats_chart, [
        "udl_widget", "param_widget", "level_widget", "start_date_widget", "end_date_widget"
    ])

    # Update plot type dropdown options
    widgets_manager.plot_type_widget.options = list(plot_manager.plots.keys())
    widgets_manager.plot_type_widget.value = list(plot_manager.plots.keys())[0]

def filter_dataframe(df, selected_udl, selected_matu, selected_param, selected_level, start_date, end_date):
    filtered_df = df.loc[start_date:end_date, (
        (df.columns.get_level_values('UDL').isin(selected_udl)) &
        (df.columns.get_level_values('Matu').isin(selected_matu)) &
        (df.columns.get_level_values('Param').isin(selected_param)) &
        (df.columns.get_level_values('Level').isin(selected_level))
    )]
    print(f"[DEBUG] Filtered DataFrame shape: {filtered_df.shape}")
    return filtered_df

def update_widget_visibility(widgets_manager, plot_manager, selected_plot):
    required_widgets = plot_manager.get_required_widgets(selected_plot)
    for widget_name in ["udl_widget", "matu_widget", "param_widget", "level_widget", "window_widget", "start_date_widget", "end_date_widget"]:
        widget = getattr(widgets_manager, widget_name)
        if widget_name in required_widgets:
            widget.layout.display = 'flex'
            widget.disabled = False
        else:
            widget.layout.display = 'none'
            widget.disabled = True

def handle_plot_button_clicked(app):
    plot_type = app.widgets.plot_type_widget.value
    plot_function = app.plot_manager.get_plot_function(plot_type)

    if plot_function is None:
        with app.output_plot:
            clear_output(wait=True)
            print(f"Error: Plot type '{plot_type}' is not registered. Please select a valid plot type.")
        return

    # Extract selected parameters
    selected_udl = list(app.widgets.udl_widget.value)
    selected_matu = [app.widgets.matu_widget.value] if isinstance(app.widgets.matu_widget, widgets.Dropdown) else list(app.widgets.matu_widget.value)
    selected_param = list(app.widgets.param_widget.value)
    selected_level = list(app.widgets.level_widget.value)
    window = app.widgets.window_widget.value
    start_date = app.widgets.start_date_widget.value
    end_date = app.widgets.end_date_widget.value

    if start_date > end_date:
        with app.output_plot:
            clear_output(wait=True)
            print("Start Date must be before End Date. Please correct your selection.")
        return

    # Filter Data
    filtered_df = app.filter_data(app.df, selected_udl, selected_matu, selected_param, selected_level, start_date, end_date)
    if filtered_df.empty:
        with app.output_plot:
            clear_output(wait=True)
            print("No valid data available for the selected combination. Please adjust your selection.")
            app.widgets.add_to_slide_button.disabled = True
        return

    # Display filtered DataFrame
    app.display_data_below(filtered_df)

    # Generate and display the plot
    with app.main_display:
        clear_output(wait=True)
        try:
            fig = plot_function(filtered_df, window) if plot_type == "Time Series Plot" else plot_function(filtered_df)
            plt.show()
            app.current_plot_data = {
                "figure": fig,
                "data": filtered_df,
                "type": plot_type,
                "position": app.widgets.position_dropdown.value
            }
            app.widgets.add_to_slide_button.disabled = False
        except Exception as e:
            print(f"An error occurred while generating the plot: {e}")

def display_dataframe_below(output_widget, df):
    with output_widget:
        clear_output(wait=True)
        display(HTML(df.to_html(max_rows=10)))

def handle_add_to_slide(app):
    if not app.current_plot_data:
        return

    position = app.widgets.position_dropdown.value
    app.current_plot_data["position"] = position

    slide = app.slide_manager.get_current_slide()

    if any(plot["position"] == position for plot in slide["plots"]):
        with app.main_display:
            clear_output(wait=True)
            print(f"Position '{position}' is already occupied. Please choose another position.")
        return

    app.slide_manager.add_plot_to_slide(app.current_plot_data.copy(), position)
    app.current_plot_data = {}
    app.switch_to_slide_view()
    app.update_sidebar()

def handle_add_slide(app):
    app.slide_manager.add_slide()
    app.update_sidebar()
    app.switch_to_slide_view()

def handle_export(app):
    prs = Presentation()
    for slide in app.slide_manager.slides:
        slide_layout = prs.slide_layouts[5]
        slide_to_add = prs.slides.add_slide(slide_layout)

        for plot_data in slide["plots"]:
            img_stream = BytesIO()
            fig = plot_data["figure"]
            fig.savefig(img_stream, format='png')
            img_stream.seek(0)
            left, top = app.get_slide_position(plot_data["position"])
            slide_to_add.shapes.add_picture(img_stream, left, top, width=Inches(5.0), height=Inches(3.75))
            plt.close(fig)

    pptx_filename = "Generated_Presentation.pptx"
    prs.save(pptx_filename)
    logging.info(f"Presentation exported as '{pptx_filename}'")

    if os.path.exists(pptx_filename):
        print(f"Presentation saved successfully as '{pptx_filename}' in the current directory.")
    else:
        print("Error: The presentation could not be saved.")

def switch_to_slide_view_helper(app):
    app.main_display.clear_output(wait=True)
    with app.main_display:
        slide = app.slide_manager.get_current_slide()
        fig, axs = plt.subplots(2, 2, figsize=(10, 8))
        for ax in axs.flatten():
            ax.clear()
            ax.axis('off')

        for plot_data in slide["plots"]:
            pos = POSITION_MAP.get(plot_data["position"], None)
            if pos is not None:
                ax = axs[pos]
                ax.axis('on')
                generate_plot_image(plot_data["figure"], ax)
                ax.set_title(plot_data["type"])
            else:
                logging.warning(f"Invalid position '{plot_data['position']}' for plot.")

        plt.tight_layout()
        plt.show()

def update_sidebar_helper(app):
    sidebar_content = []
    for i, slide in enumerate(app.slide_manager.slides):
        fig, axs = plt.subplots(2, 2, figsize=(4, 4))
        for ax in axs.flatten():
            ax.clear()
            ax.axis('off')

        for plot_data in slide["plots"]:
            pos = POSITION_MAP.get(plot_data["position"], None)
            if pos is not None:
                ax = axs[pos]
                ax.axis('on')
                generate_plot_image(plot_data["figure"], ax)

        plt.tight_layout()

        buf = BytesIO()
        fig.savefig(buf, format="png")
        plt.close(fig)
        buf.seek(0)
        img = Image.open(buf)
        img.thumbnail(THUMBNAIL_SIZE)

        with BytesIO() as output:
            img.save(output, format="PNG")
            img_widget = widgets.Image(value=output.getvalue(), format='png', width=THUMBNAIL_SIZE[0], height=THUMBNAIL_SIZE[1])

        slide_label = widgets.Label(f"Slide {i + 1}")
        slide_button = widgets.VBox([img_widget, slide_label], layout=widgets.Layout(
            width="150px", height="150px", border="1px solid black", align_items="center", padding="5px"
        ))

        slide_button_box = widgets.Button(description=f"Select Slide {i + 1}", button_style='info', layout=widgets.Layout(width="150px", margin="5px 0"))
        slide_button_box.on_click(lambda b, idx=i: app.select_slide(idx))

        sidebar_content.append(widgets.VBox([slide_button, slide_button_box]))

    app.left_sidebar.children = sidebar_content

### APP

In [19]:
class App:
    def __init__(self, df):
        # Data and Managers Initialization
        self.df = df
        self.widgets = WidgetManager(df)
        self.plot_manager = PlotManager()
        self.slide_manager = SlideManager()
        self.current_plot_data = {}

        # Layout Outputs
        self.df_below_output = create_output_widget(width="30%", border="1px solid #d3d3d3", padding="-100px", margin="0 auto")
        self.output_plot = widgets.Output()
        self.main_display = create_output_widget(width="70%", height="500px", border="1px solid black", padding="0px", margin="0 auto")
        self.left_sidebar = create_sidebar_widget(width="18%", border="1px solid #d3d3d3", padding="10px", background_color="#f9f9f9", margin="0 10px 0 0")

        # Register Plot Functions
        self.register_plots()

        # Build Layout
        self.customization_window = LayoutBuilder.create_customization_window(self.widgets)
        self.build_layout()

        # Bind Widget Events
        self.bind_events()

        # Initial Sidebar Update
        self.update_sidebar()

    ########## Modularize Layout Construction ##########
    def build_layout(self):
        # Build the entire layout for the app
        main_layout = LayoutBuilder.create_main_layout(
            self.left_sidebar, self.main_display, self.customization_window, self.df_below_output
        )
        display(main_layout)
        print("[DEBUG] Layout built and displayed.")

    ########## Widget Event Binding ##########
    def bind_events(self):
        # Bind the necessary event handlers to widgets
        bind_widget_events(self.widgets, {
            'plot_type_widget': self.on_plot_type_change,
            'plot_button': self.on_plot_button_clicked,
            'add_to_slide_button': self.on_add_to_slide_button_clicked,
            'add_slide_button': self.on_add_slide_button_clicked,
            'export_button': self.on_export_button_clicked
        })
        print("[DEBUG] Events bound to widgets.")

    ########## Register Plot Functions ##########
    def register_plots(self):
        # Registering available plots
        register_available_plots(self.plot_manager, self.widgets)
        print("[DEBUG] Plots registered and plot type widget options set.")

    ########## Encapsulate Data Filtering ##########
    def filter_data(self, df, selected_udl, selected_matu, selected_param, selected_level, start_date, end_date):
        # Filter data based on selected parameters
        print("[DEBUG] Filtering data with the following parameters:")
        print(f"UDL: {selected_udl}, Matu: {selected_matu}, Param: {selected_param}, Level: {selected_level}, Start: {start_date}, End: {end_date}")
        return filter_dataframe(df, selected_udl, selected_matu, selected_param, selected_level, start_date, end_date)

    ########## Event Handlers ##########
    def on_plot_type_change(self, change):
        print("[DEBUG] Plot type changed.")
        update_widget_visibility(self.widgets, self.plot_manager, change['new'])

    def on_plot_button_clicked(self, b):
        print("[DEBUG] Plot button clicked.")
        handle_plot_button_clicked(self)

    def display_data_below(self, df):
        # Display the filtered dataframe below the main view without clearing the existing content
        print("[DEBUG] Displaying data below customization window.")
        display_dataframe_below(self.df_below_output, df)

    def on_add_to_slide_button_clicked(self, b):
        print("[DEBUG] Add to slide button clicked.")
        handle_add_to_slide(self)

    def on_add_slide_button_clicked(self, b):
        print("[DEBUG] Add slide button clicked.")
        handle_add_slide(self)

    def on_export_button_clicked(self, b):
        print("[DEBUG] Export button clicked.")
        handle_export(self)

    def switch_to_slide_view(self):
        print("[DEBUG] Switching to slide view.")
        switch_to_slide_view_helper(self)

    def update_sidebar(self):
        print("[DEBUG] Updating sidebar.")
        update_sidebar_helper(self)

    def select_slide(self, index):
        print(f"[DEBUG] Selecting slide {index + 1}.")
        self.slide_manager.selected_slide_index = index
        self.switch_to_slide_view()

# Inputs

## Configuration constants

In [20]:
FIGURE_SIZE = (5, 4)
THUMBNAIL_SIZE = (120, 120)
POSITION_MAP = {
    "top-left": (0, 0),
    "top-right": (0, 1),
    "bottom-left": (1, 0),
    "bottom-right": (1, 1)
}
THUMBNAIL_SIZE = (120, 120)

# Setup logging
logging.basicConfig(level=logging.INFO)

## Example usage

In [16]:
udl = ['US_SPX', 'FR_CAC', 'DE_DAX', 'ES_IBEX']
matu = ['None', 1, 2, 3, 6, 12, 24]
param = ['Spot', 'Delta', 'Moneyness']
level = {
    'Spot': ['None'],
    'Delta': [5, 10, 15, 25, 35, 45, 50, 55, 65, 75, 86, 90, 95],
    'Moneyness': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
}
df = create_multi_index_dataframe(udl, matu, param, level)

# Run

In [17]:
app = App(df)

[DEBUG] Plots registered and plot type widget options set.


[DEBUG] Layout built and displayed.
[DEBUG] Events bound to widgets.
[DEBUG] Updating sidebar.
